In [ ]:
%matplotlib inline
import matplotlib.image as mpimg
import numpy as np
import matplotlib.pyplot as plt
import os,sys
from PIL import Image
from helpers import *

In [ ]:
# Loaded a set of images
root_dir = "training/"

image_dir = root_dir + "images/"
files = os.listdir(image_dir)
n = len(files)
print("Loading " + str(n) + " images")
imgs = [load_image(image_dir + files[i]) for i in range(n)]
print(files[0])

gt_dir = root_dir + "groundtruth/"
print("Loading " + str(n) + " images")
gt_imgs = [load_image(gt_dir + files[i]) for i in range(n)]

In [ ]:
# Extract patches from input images
patch_size = 16 # each patch is 16*16 pixels

img_patches = [img_crop(imgs[i], patch_size, patch_size) for i in range(n)]
gt_patches = [img_crop(gt_imgs[i], patch_size, patch_size) for i in range(n)]

# Linearize list of patches
img_patches = np.asarray([img_patches[i][j] for i in range(len(img_patches)) for j in range(len(img_patches[i]))])
gt_patches =  np.asarray([gt_patches[i][j] for i in range(len(gt_patches)) for j in range(len(gt_patches[i]))])

print(img_patches.shape)
print(gt_patches.shape)

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, ReLU, Dropout, Flatten, Dense, ELU
from keras.regularizers import l2

INPUT_SHAPE=(16, 16, 3)
REG = 1e-6

model = Sequential()
model.add(Conv2D(filters = 64,
                 kernel_size = (5, 5),
                 strides = (1, 1),
                 padding='same',
                 input_shape=INPUT_SHAPE))
model.add(ELU())
#model.add(MaxPooling2D(pool_size=(2, 2),
#                       strides=None,
#                       padding='same'))
model.add(Conv2D(filters = 64,
                 kernel_size = (5, 5),
                 strides = (2, 2),
                 padding='same',
                 input_shape=INPUT_SHAPE))
#model.add(Dropout(0.25))
model.add(Conv2D(filters = 128,
                 kernel_size = (4, 4),
                 strides = (1, 1),
                 padding='same',
                 input_shape=INPUT_SHAPE))
model.add(ELU())
#model.add(ReLU(max_value=2))
#model.add(MaxPooling2D(pool_size=(2, 2),
#                       strides=None,
#                       padding='same'))
model.add(Conv2D(filters = 128,
                 kernel_size = (4, 4),
                 strides = (2, 2),
                 padding='same',
                 input_shape=INPUT_SHAPE))
#model.add(Dropout(0.25))
model.add(Conv2D(filters = 256,
                 kernel_size = (3, 3),
                 strides = (1, 1),
                 padding='same',
                 input_shape=INPUT_SHAPE))
model.add(ELU())
#model.add(ReLU(max_value=2))
#model.add(MaxPooling2D(pool_size=(2, 2),
#                       strides=None,
#                       padding='same'))
model.add(Conv2D(filters = 256,
                 kernel_size = (3, 3),
                 strides = (2, 2),
                 padding='same',
                 input_shape=INPUT_SHAPE))
model.add(Flatten())
model.add(Dense(128, kernel_regularizer=l2(REG)))
model.add(ELU())
#model.add(ReLU(max_value=2))
model.add(Dropout(0.25))
model.add(Dense(64, kernel_regularizer=l2(REG)))
model.add(ELU())
#model.add(ReLU(max_value=2))
model.add(Dropout(0.5))
model.add(Dense(2, kernel_regularizer=l2(REG)))

print(model.summary())

In [ ]:
from keras.optimizers import Adam

opt = Adam(lr=0.001) # Adam optimizer with default initial learning rate
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [ ]:
foreground_threshold = 0.25 # percentage of pixels > 1 required to assign a foreground label to a patch

def value_to_class(v):
    df = np.sum(v)
    if df > foreground_threshold:
        return [1, 0]
    else:
        return [0, 1]

X = np.asarray([ img_patches[i] for i in range(len(img_patches))])
Y = np.asarray([value_to_class(np.mean(gt_patches[i])) for i in range(len(gt_patches))])
print(X.shape)
print(Y.shape)

In [ ]:
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator

STEPS_PER_EPOCH = 125
BATCH_SIZE = 125
EPOCHS = 200

datagen = ImageDataGenerator(zca_whitening=True,
                             rotation_range=90,
                             horizontal_flip=True,
                             vertical_flip=True)

datagen.fit(X)

In [ ]:
model.fit_generator(datagen.flow(X, Y, batch_size=BATCH_SIZE),
          steps_per_epoch=X.shape[0] // BATCH_SIZE,
          epochs=EPOCHS,
          verbose=2)